In [ ]:

############ Load libraries  ############

import numpy as np
import sys
from PIL import Image
import torch
from glob import glob
from ultralytics import YOLO
import pandas as pd
from tqdm import tqdm
from torch import nn
from torch.nn import functional as F
from torchvision import transforms as T
from torchvision import transforms, datasets, models
tqdm.pandas()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device='cuda'

In [ ]:

############ Preprocessing and Trasnforms ############

imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transform = T.Compose([T.Resize(224),
                       T.ToTensor(),
                       T.Normalize(*imagenet_stats)])

In [ ]:

############ Gender Classification ############

class EffModel(torch.nn.Module):
    def __init__(self,num_classes=1000):
        super(EffModel, self).__init__()
        self.model = models.efficientnet_b0(weights=None)
        self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, 1)

    def forward(self, x):
        x = self.model(x)
        x = F.sigmoid(x)
        return x


ckpt = torch.load('gender_effb0_body.pth', map_location=device)
gender = EffModel(num_classes=1)
gender.load_state_dict(ckpt)
gender.to(device)
gender.eval()

print('gender model loaded')


gender model loaded


In [ ]:

############ Location Classification ############

class ResnetModel(nn.Module):
    def __init__(self,num_classes=1000):
        super(ResnetModel, self).__init__()
        self.model = models.resnet50(weights=None)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = F.softmax(x)
        return x

ckpt = torch.load('resnet50_fined.pt', map_location=device)
classifier = ResnetModel(num_classes=488)
classifier.load_state_dict(ckpt)
classifier.eval()
classifier.to(device)

print('loc classifier model loaded')

loc classifier model loaded


In [ ]:

############ Body Detection ############

yolo_model = YOLO('yolov8n.pt',)
yolo_model.to(device)
dict_classes = yolo_model.model.names
class_IDS = [0]

print('detection model loaded')

detection model loaded


In [ ]:

############ Body Detection, Gender Classification, Location Classification ############

probas = []
males = []
females = []
paths = []

test_path = '/media/nextera/New Volume/facecup/Facecup/test/*'
files = glob(test_path)
for filename in tqdm(files):
    cnt = 0
    ma  = 0
    fe  = 0

    img = Image.open(filename).convert('RGB')

    out = yolo_model.predict(img,verbose=False ,conf=0.2,classes =[0])
    for bb in out[0].boxes.xyxy:
        cnt += 1
        bb = bb.detach().cpu().numpy().astype(int)
        imm = img.crop([bb[0],bb[1],bb[2],bb[3]])

        imm = transform(imm)
        gen_pred = gender(imm.unsqueeze(0).to(device)).detach().cpu().numpy()[0][0]

        if gen_pred<0.4:
            ma+=1
        else:
            fe+=1

    males.append(ma)
    females.append(fe)
    img = transform(img)
    probas.append(classifier(img.unsqueeze(0).to(device)).detach().cpu().numpy()[0])
    paths.append(filename.split('/')[-1])


  0%|                                                  | 0/1912 [00:00<?, ?it/s]2024-11-21 12:45:31.377967: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 12:45:31.377988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 12:45:31.378599: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_105758/3147962292.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)
100%|███████████████████████████████████████| 1912/1912 [02:42<00:00, 11.74it/s]


In [ ]:
sub = pd.DataFrame({'path':paths,'males':males,'females':females,'probas':probas})

In [ ]:

############ create submission file  ############

temp = sub.probas.apply(pd.Series)
sub = pd.concat([sub,temp],axis=1)
sub = sub.drop(['probas'],axis=1)

In [ ]:
sub.to_csv('submission.csv',index=False)

In [ ]:
sub

,path,males,females,0,1,2,3,4,5,6,...,478,479,480,481,482,483,484,485,486,487
0,0277ZM44Eq.jpg,0,0,1.760927e-10,5.173466e-12,3.189623e-11,3.942789e-09,1.857893e-09,1.783455e-07,4.158910e-11,...,2.496867e-12,2.840210e-09,5.286805e-16,3.711946e-18,1.933043e-08,3.198585e-11,2.898930e-09,2.370383e-13,7.002500e-13,1.321872e-08
1,05ka6s44sS.jpg,0,0,2.165855e-09,5.411411e-05,3.062753e-05,1.217953e-01,8.582868e-06,5.251305e-07,3.540143e-06,...,4.544790e-05,3.136132e-08,2.813950e-04,1.036873e-11,1.240104e-04,7.469746e-11,1.074850e-02,1.079028e-10,2.133428e-04,5.685300e-06
2,07e2AzEYtZ.jpg,0,0,7.094311e-08,8.114763e-08,1.325965e-06,1.387383e-08,1.677370e-06,1.205641e-06,6.626371e-07,...,1.423586e-07,2.889756e-08,1.628471e-13,4.684622e-14,1.484461e-03,4.578920e-06,3.620041e-06,2.552888e-08,2.290393e-07,1.750373e-09
3,07Jpl0mW4u.jpg,7,0,5.065554e-10,1.130840e-10,7.868045e-13,2.909860e-12,4.467256e-11,5.939565e-13,8.706084e-11,...,1.134508e-07,3.330230e-10,2.024220e-11,1.959546e-09,2.314606e-10,2.728715e-12,1.034137e-11,1.006946e-09,2.598137e-09,1.942181e-12
4,0CzwVbIsZc.jpg,3,1,5.477203e-09,5.228209e-09,1.118849e-07,4.245859e-06,3.928066e-10,3.743994e-09,3.321281e-05,...,2.633964e-08,9.436852e-06,9.067673e-04,7.843987e-11,6.366465e-09,1.670897e-07,2.126501e-04,6.304627e-14,4.575912e-09,3.922021e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907,Zw4z0Q7wyK.jpg,0,0,2.454481e-10,1.172665e-11,2.112441e-15,1.270205e-13,3.190281e-13,4.254857e-10,2.667991e-09,...,4.219109e-12,3.744367e-12,3.868274e-17,5.045406e-19,6.553785e-11,1.057347e-10,1.317670e-11,3.441107e-16,3.642948e-14,2.109483e-10
1908,ZwxAVzHKFj.jpg,0,0,8.623428e-13,1.171546e-05,1.618085e-14,5.710765e-08,4.130115e-13,1.720171e-10,3.087229e-10,...,2.776657e-07,1.674595e-08,1.023367e-07,1.449879e-13,2.509636e-10,2.804660e-11,1.368552e-11,2.916379e-15,5.612972e-11,1.343661e-09
1909,ZxRTuPzJ1p.jpg,4,0,4.241613e-11,1.417271e-07,2.103488e-09,3.667786e-11,1.204878e-10,1.077347e-12,2.241257e-10,...,6.040583e-12,3.773421e-09,4.604854e-13,2.944990e-08,2.128725e-10,1.237319e-07,2.542457e-12,1.994845e-11,1.175306e-07,3.352535e-13
1910,zy5kyxwWAB.jpg,1,0,5.476573e-09,8.898303e-07,8.794539e-06,2.256692e-05,2.794949e-07,3.141856e-03,3.459585e-04,...,2.004796e-07,2.078028e-07,7.144936e-09,4.947390e-12,1.790927e-03,2.896846e-07,1.639649e-03,1.368225e-10,1.103833e-06,2.808464e-07
